In [ ]:
%cd /content
from google.colab import drive
drive.mount('/content/drive')

/content
Mounted at /content/drive


In [ ]:
#from svgpathtools import svg2paths, wsvg
import numpy as np
import cv2
import random
import json
import shapely
from shapely.geometry import Polygon
import base64
import os


In [ ]:
#u
def update_polygons(x_offset, y_offset, objects_dict, image_dict):
    for obj in objects_dict['shapes']:
        
        object_shifting(obj, x_offset, y_offset) #updating object mask coordinates
        image_dict['shapes'].append(obj) #adding mask to image annotation
        
        #checking each object originally existing on image for an intersection
        for obj2 in image_dict['shapes']: 
            if Polygon(obj2['points']).intersects(Polygon(obj['points'])): #making polygons out of inserted and old object and check if they intersect
                new_polygon=Polygon(obj2['points']).difference(Polygon(obj['points'])) #making new polygon out of old object, subtracting the area, overlayed by new object
                if type(new_polygon) is Polygon and not new_polygon.is_empty: #checking if new polygon is not empty and not disjoint to separate polygons
                    obj2['points']=list(zip(*new_polygon.exterior.coords.xy)) #saving polygon coordinates to old object mask annotaion
                else: #deleting empty or disjoint polygons
                    del obj2
                #obj2['points']=list(zip(*new_polygon.exterior.coords.xy))
                #obj2['points']=np.array(new_polygon).tolist()
                #obj2['points']=list(new_polygon.exterior.coords)

#function updating the inserted objects mask coordinates, to fit new position on bitmap
def object_shifting(obj, x_offset, y_offset):
    for point in obj['points']:
        point[0]+=x_offset
        point[1]+=y_offset
        
#main function for object insertion
def insert_objects(img, num):
    #opening a random object
    ob=['287','457','1193','394','000000012993','747'] 
    object_number=random.randrange(0, len(ob))
    obj = cv2.imread("/content/drive/MyDrive/obj/{object_number}.png".format(object_number=ob[object_number]), -1) #opening an object 
    with open("/content/drive/MyDrive/obj/{object_number}.json".format(object_number=ob[object_number]), "r") as infile1:  #opening object annotation
        json_obj=json.load(infile1)
    #print(object_number,img.shape,obj.shape)
    x_offset=random.randrange(50, img.shape[1]-obj.shape[1]) #randomly selected distance between image and object left edges 
    y_offset=random.randrange(150, img.shape[0]-obj.shape[0]-200) #randomly selected distance between image and object top edges 
    y1, y2 = y_offset, y_offset + obj.shape[0] #coordinates of the box where object is inserted
    x1, x2 = x_offset, x_offset + obj.shape[1]
    alpha_s = obj[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s #transparent pixel coeffcient
    #print(x_offset, y_offset)
    #updating image pixels values (rewriting them to objecct pixels)
    for c in range(0, 3):
        img[y1:y2, x1:x2, c] = (alpha_s * obj[:, :, c] +
                              alpha_l * img[y1:y2, x1:x2, c])   #updating image pixels values (rewriting them to objecct pixels)  
    
    #saving new image
    cv2.imwrite('/content/drive/MyDrive/yolact-master/data/coco/new/new_{num}.png'.format(num=num), img) #savin new image
    
    #opening image annotaion
    with open("/content/drive/MyDrive/yolact-master/data/coco/testes/{im}.json".format(im=num), "r") as infile1: 
        json_img=json.load(infile1)
        
    #opening image annotaion
    with open("/content/drive/MyDrive/obj/{im}.json".format(im=ob[object_number]), "r") as infile: 
        json_obj=json.load(infile)

    
    update_polygons(x_offset, y_offset, json_obj, json_img)
    #updating path to file in annotation
    json_img['imagePath']='new_{num}.png'.format(num=num)

    #updating image itself, encoded into json (needed only for labelme) 
    #for some reason, not working, even while labelme using same function for encoding
    json_img['imageData']= bytes.decode(base64.b64encode(open('/content/drive/MyDrive/yolact-master/data/coco/new/new_{num}.png'.format(num=num), "rb").read()))

    #saving new file annotation
    with open('/content/drive/MyDrive/yolact-master/data/coco/new/new_{num}.json'.format(num=num), "w") as outfile: 
        json.dump(json_img, outfile,sort_keys=True,indent=4)
    


In [ ]:
OBJECTS_PER_PICTURE=1 #set how many objects should be added to a picture
"""first_image=0 #151
last_image=150#1201
step=5"""
end="png"
folder="/content/drive/MyDrive/yolact-master/data/coco/testes/"
allfile=os.listdir(folder)
for image in allfile:
#range(first_image, last_image+1, step): #iterating through labelled images
  if image.endswith(end):
    image=image.split(".")
    #print(image)
    #image=re.split(r'(\d.*)',image[0])
    #print(image)
    #image=int(image[1])
    image=image[0]
    #print(image)
    path="/content/drive/MyDrive/yolact-master/data/coco/testes/{im}.png".format(im=image)
    img = cv2.imread(path)
    for objects in range(OBJECTS_PER_PICTURE): #inserting objects into images
        insert_objects(img, image)
   

In [ ]:
cv2.destroyAllWindows()